In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
df=pd.read_csv('train.csv')

In [3]:
new_df=df.sample(30000,random_state=2)

In [4]:
new_df.shape

(30000, 6)

In [5]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
398782,398782,496695,532029,What is the best marketing automation tool for...,What is the best marketing automation tool for...,1
115086,115086,187729,187730,I am poor but I want to invest. What should I do?,I am quite poor and I want to be very rich. Wh...,0
327711,327711,454161,454162,I am from India and live abroad. I met a guy f...,T.I.E.T to Thapar University to Thapar Univers...,0
367788,367788,498109,491396,Why do so many people in the U.S. hate the sou...,My boyfriend doesnt feel guilty when he hurts ...,0
151235,151235,237843,50930,Consequences of Bhopal gas tragedy?,What was the reason behind the Bhopal gas trag...,0


In [6]:
from bs4 import BeautifulSoup
import re
import warnings
warnings.filterwarnings('ignore')

In [7]:
def preprocess(ques):
    ques = str(ques).lower().strip()

    #Replacing some of the characters 
    ques=ques.replace('@', 'at')
    ques=ques.replace('$', 'dollar')
    ques=ques.replace('%', 'percent')
    ques=ques.replace('&', 'and')
    ques=ques.replace('=', 'equals')

    ques=ques.replace('[math]','') # math occurs around 900 times in the whole dataset and means nothing

    contractions = { 
    
                "ain't": "am not / are not / is not / has not / have not",
                "aren't": "are not / am not",
                "can't": "cannot",
                "can't've": "cannot have",
                "'cause": "because",
                "could've": "could have",
                "couldn't": "could not",
                "couldn't've": "could not have",
                "didn't": "did not",
                "doesn't": "does not",
                "don't": "do not",
                "hadn't": "had not",
                "hadn't've": "had not have",
                "hasn't": "has not",
                "haven't": "have not",
                "he'd": "he had / he would",
                "he'd've": "he would have",
                "he'll": "he shall / he will",
                "he'll've": "he shall have / he will have",
                "he's": "he has / he is",
                "how'd": "how did",
                "how'd'y": "how do you",
                "how'll": "how will",
                "how's": "how has / how is / how does",
                "I'd": "I had / I would",
                "I'd've": "I would have",
                "I'll": "I shall / I will",
                "I'll've": "I shall have / I will have",
                "I'm": "I am",
                "I've": "I have",
                "isn't": "is not",
                "it'd": "it had / it would",
                "it'd've": "it would have",
                "it'll": "it shall / it will",
                "it'll've": "it shall have / it will have",
                "it's": "it has / it is",
                "let's": "let us",
                "ma'am": "madam",
                "mayn't": "may not",
                "might've": "might have",
                "mightn't": "might not",
                "mightn't've": "might not have",
                "must've": "must have",
                "mustn't": "must not",
                "mustn't've": "must not have",
                "needn't": "need not",
                "needn't've": "need not have",
                "o'clock": "of the clock",
                "oughtn't": "ought not",
                "oughtn't've": "ought not have",
                "shan't": "shall not",
                "sha'n't": "shall not",
                "shan't've": "shall not have",
                "she'd": "she had / she would",
                "she'd've": "she would have",
                "she'll": "she shall / she will",
                "she'll've": "she shall have / she will have",
                "she's": "she has / she is",
                "should've": "should have",
                "shouldn't": "should not",
                "shouldn't've": "should not have",
                "so've": "so have",
                "so's": "so as / so is",
                "that'd": "that would / that had",
                "that'd've": "that would have",
                "that's": "that has / that is",
                "there'd": "there had / there would",
                "there'd've": "there would have",
                "there's": "there has / there is",
                "they'd": "they had / they would",
                "they'd've": "they would have",
                "they'll": "they shall / they will",
                "they'll've": "they shall have / they will have",
                "they're": "they are",
                "they've": "they have",
                "to've": "to have",
                "wasn't": "was not",
                "we'd": "we had / we would",
                "we'd've": "we would have",
                "we'll": "we will",
                "we'll've": "we will have",
                "we're": "we are",
                "we've": "we have",
                "weren't": "were not",
                "what'll": "what shall / what will",
                "what'll've": "what shall have / what will have",
                "what're": "what are",
                "what's": "what has / what is",
                "what've": "what have",
                "when's": "when has / when is",
                "when've": "when have",
                "where'd": "where did",
                "where's": "where has / where is",
                "where've": "where have",
                "who'll": "who shall / who will",
                "who'll've": "who shall have / who will have",
                "who's": "who has / who is",
                "who've": "who have",
                "why's": "why has / why is",
                "why've": "why have",
                "will've": "will have",
                "won't": "will not",
                "won't've": "will not have",
                "would've": "would have",
                "wouldn't": "would not",
                "wouldn't've": "would not have",
                "y'all": "you all",
                "y'all'd": "you all would",
                "y'all'd've": "you all would have",
                "y'all're": "you all are",
                "y'all've": "you all have",
                "you'd": "you had / you would",
                "you'd've": "you would have",
                "you'll": "you shall / you will",
                "you'll've": "you shall have / you will have",
                "you're": "you are",
                "you've": "you have"
                }
    ques_deconstruct=[] 
    for word in ques.split():
        if word in contractions:
            word=contractions[word]
            
        ques_deconstruct.append(word)

    ques = ' '.join(ques_deconstruct)
    ques = ques.replace("ve","have")
    ques = ques.replace("re","are")
    ques = ques.replace("'ll","will")
    ques = ques.replace("n't","not")

    #Remove html tags using beautifulsoup
    ques = BeautifulSoup(ques)
    ques = ques.get_text()

     #Removing all the punctuations from the dataset
    punc = re.compile('\W')
    ques=re.sub(punc, ' ',ques).strip()

  
    return ques

In [8]:
new_df['question1'] = new_df['question1'].apply(preprocess)
new_df['question2'] = new_df['question2'].apply(preprocess)

In [9]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1
115086,115086,187729,187730,i am poor but i want to inhavest what should ...,i am quite poor and i want to be havery rich ...,0
327711,327711,454161,454162,i am from india and lihave abroad i met a guy...,t i e t to thapar unihaversity to thapar uniha...,0
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the areason behind the bhopal gas tra...,0


In [10]:
new_df['q1_len'] = new_df['question1'].str.len()
new_df['q2_len'] = new_df['question2'].str.len()

In [11]:
new_df['q1_words'] = new_df['question1'].apply(lambda row: len(row.split(" ")))
new_df['q2 _words'] = new_df['question2'].apply(lambda row: len(row.split(" ")))

In [12]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_words,q2 _words
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,75,76,13,13
115086,115086,187729,187730,i am poor but i want to inhavest what should ...,i am quite poor and i want to be havery rich ...,0,50,58,13,16
327711,327711,454161,454162,i am from india and lihave abroad i met a guy...,t i e t to thapar unihaversity to thapar uniha...,0,106,123,28,21
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,58,145,14,32
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the areason behind the bhopal gas tra...,0,34,50,5,9


In [13]:
def common_Words(row):
    w1 = set(map(lambda word : word.lower().strip(), row['question1'].split(" "))) # Finding common words in q1 and q2
    w2 = set(map(lambda word : word.lower().strip(), row['question2'].split(" ")))
    return len(w1&w2)
    

In [14]:
new_df['common_words'] = new_df.apply(common_Words,axis =1)

In [15]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_words,q2 _words,common_words
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,75,76,13,13,12
115086,115086,187729,187730,i am poor but i want to inhavest what should ...,i am quite poor and i want to be havery rich ...,0,50,58,13,16,8
327711,327711,454161,454162,i am from india and lihave abroad i met a guy...,t i e t to thapar unihaversity to thapar uniha...,0,106,123,28,21,4
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,58,145,14,32,1
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the areason behind the bhopal gas tra...,0,34,50,5,9,3


In [16]:
def total_words(row):
    w1 = set(map(lambda word : word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word : word.lower().strip(), row['question2'].split(" ")))
    return (len(w1)+len(w2))
    

In [17]:
new_df['ttl_words'] = new_df.apply(total_words,axis=1)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_words,q2 _words,common_words,ttl_words
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,75,76,13,13,12,26
115086,115086,187729,187730,i am poor but i want to inhavest what should ...,i am quite poor and i want to be havery rich ...,0,50,58,13,16,8,24
327711,327711,454161,454162,i am from india and lihave abroad i met a guy...,t i e t to thapar unihaversity to thapar uniha...,0,106,123,28,21,4,38
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,58,145,14,32,1,34
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the areason behind the bhopal gas tra...,0,34,50,5,9,3,13


In [18]:
new_df['word_share'] = round(new_df['common_words'] / new_df['ttl_words'],3) # Here 3 means round off the value upto 3 decimal places

In [19]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_words,q2 _words,common_words,ttl_words,word_share
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,75,76,13,13,12,26,0.462
115086,115086,187729,187730,i am poor but i want to inhavest what should ...,i am quite poor and i want to be havery rich ...,0,50,58,13,16,8,24,0.333
327711,327711,454161,454162,i am from india and lihave abroad i met a guy...,t i e t to thapar unihaversity to thapar uniha...,0,106,123,28,21,4,38,0.105
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,58,145,14,32,1,34,0.029
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the areason behind the bhopal gas tra...,0,34,50,5,9,3,13,0.231


In [20]:
# Advanced Features
from nltk.corpus import stopwords

def fetch_token_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    SAFE_DIV = 0.0001 

    STOP_WORDS = stopwords.words("english")
    
    token_features = [0.0]*8
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features

    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    return token_features

In [21]:
token_features = new_df.apply(fetch_token_features, axis=1)

new_df["cwc_min"]       = list(map(lambda x: x[0], token_features))
new_df["cwc_max"]       = list(map(lambda x: x[1], token_features))
new_df["csc_min"]       = list(map(lambda x: x[2], token_features))
new_df["csc_max"]       = list(map(lambda x: x[3], token_features))
new_df["ctc_min"]       = list(map(lambda x: x[4], token_features))
new_df["ctc_max"]       = list(map(lambda x: x[5], token_features))
new_df["last_word_eq"]  = list(map(lambda x: x[6], token_features))
new_df["first_word_eq"] = list(map(lambda x: x[7], token_features))

In [22]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_words,q2 _words,...,ttl_words,word_share,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,75,76,13,13,...,26,0.462,0.874989,0.874989,0.999980,0.999980,0.923070,0.923070,1.0,1.0
115086,115086,187729,187730,i am poor but i want to inhavest what should ...,i am quite poor and i want to be havery rich ...,0,50,58,13,16,...,24,0.333,0.666644,0.399992,0.714276,0.714276,0.583328,0.466664,1.0,1.0
327711,327711,454161,454162,i am from india and lihave abroad i met a guy...,t i e t to thapar unihaversity to thapar uniha...,0,106,123,28,21,...,38,0.105,0.000000,0.000000,0.428565,0.272725,0.149999,0.115384,0.0,0.0
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,58,145,14,32,...,34,0.029,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the areason behind the bhopal gas tra...,0,34,50,5,9,...,13,0.231,0.749981,0.599988,0.000000,0.000000,0.599988,0.333330,1.0,0.0


In [23]:
import distance

def fetch_length_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    length_features = [0.0]*3
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()
    
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return length_features
    
    # Absolute length features
    length_features[0] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    length_features[1] = (len(q1_tokens) + len(q2_tokens))/2
    
    strs = list(distance.lcsubstrings(q1, q2))
    if strs:
        length_features[2] = len(strs[0]) / (min(len(q1), len(q2)) + 1)
    else:
        length_features[2] = 0.0
    
    return length_features
    

In [24]:
length_features = new_df.apply(fetch_length_features, axis=1)



In [25]:
new_df['abs_len_diff'] = list(map(lambda x: x[0], length_features))
new_df['mean_len'] = list(map(lambda x: x[1], length_features))
new_df['longest_substr_ratio'] = list(map(lambda x: x[2], length_features))

In [26]:
from fuzzywuzzy import fuzz

def fetch_fuzzy_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    fuzzy_features = [0.0]*4
    
    # fuzz_ratio
    fuzzy_features[0] = fuzz.QRatio(q1, q2)

    # fuzz_partial_ratio
    fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

    # token_sort_ratio
    fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

    # token_set_ratio
    fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

    return fuzzy_features

In [27]:
fuzzy_features = new_df.apply(fetch_fuzzy_features, axis=1)

# Creating new feature columns for fuzzy features
new_df['fuzz_ratio'] = list(map(lambda x: x[0], fuzzy_features))
new_df['fuzz_partial_ratio'] = list(map(lambda x: x[1], fuzzy_features))
new_df['token_sort_ratio'] = list(map(lambda x: x[2], fuzzy_features))
new_df['token_set_ratio'] = list(map(lambda x: x[3], fuzzy_features))

In [28]:
print(new_df.shape)
new_df.head()

(30000, 28)


,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_words,q2 _words,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,75,76,13,13,...,0.923070,1.0,1.0,0.0,13.0,0.855263,99,99,99,99
115086,115086,187729,187730,i am poor but i want to inhavest what should ...,i am quite poor and i want to be havery rich ...,0,50,58,13,16,...,0.466664,1.0,1.0,3.0,13.5,0.215686,70,68,62,74
327711,327711,454161,454162,i am from india and lihave abroad i met a guy...,t i e t to thapar unihaversity to thapar uniha...,0,106,123,28,21,...,0.115384,0.0,0.0,6.0,23.0,0.046729,26,30,32,38
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,58,145,14,32,...,0.000000,0.0,0.0,17.0,21.5,0.050847,29,41,23,30
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the areason behind the bhopal gas tra...,0,34,50,5,9,...,0.333330,1.0,0.0,4.0,7.0,0.542857,55,70,45,69


In [29]:

from sklearn.preprocessing import MinMaxScaler

X = MinMaxScaler().fit_transform(new_df[['cwc_min', 'cwc_max', 'csc_min', 'csc_max' , 'ctc_min' , 'ctc_max' , 'last_word_eq', 'first_word_eq' , 'abs_len_diff' , 'mean_len' , 'token_set_ratio' , 'token_sort_ratio' ,  'fuzz_ratio' , 'fuzz_partial_ratio' , 'longest_substr_ratio']])
y = new_df['is_duplicate'].values

In [30]:
from sklearn.manifold import TSNE
tsne3d = TSNE(
    n_components=3,
    init='random', # pca
    random_state=101,
    method='barnes_hut',
    n_iter=1000,
    verbose=2,
    angle=0.5
).fit_transform(X)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 30000 samples in 0.135s...
[t-SNE] Computed neighbors for 30000 samples in 13.944s...
[t-SNE] Computed conditional probabilities for sample 1000 / 30000
[t-SNE] Computed conditional probabilities for sample 2000 / 30000
[t-SNE] Computed conditional probabilities for sample 3000 / 30000
[t-SNE] Computed conditional probabilities for sample 4000 / 30000
[t-SNE] Computed conditional probabilities for sample 5000 / 30000
[t-SNE] Computed conditional probabilities for sample 6000 / 30000
[t-SNE] Computed conditional probabilities for sample 7000 / 30000
[t-SNE] Computed conditional probabilities for sample 8000 / 30000
[t-SNE] Computed conditional probabilities for sample 9000 / 30000
[t-SNE] Computed conditional probabilities for sample 10000 / 30000
[t-SNE] Computed conditional probabilities for sample 11000 / 30000
[t-SNE] Computed conditional probabilities for sample 12000 / 30000
[t-SNE] Computed conditional probabilities for sa

In [31]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
398782,what is the best marketing automation tool for...,what is the best marketing automation tool for...
115086,i am poor but i want to inhavest what should ...,i am quite poor and i want to be havery rich ...
327711,i am from india and lihave abroad i met a guy...,t i e t to thapar unihaversity to thapar uniha...
367788,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...
151235,consequences of bhopal gas tragedy,what was the areason behind the bhopal gas tra...


In [32]:
final_df = new_df.drop(columns=['id','qid1','qid2','question1','question2'])
print(final_df.shape)
final_df.head()

(30000, 23)


,is_duplicate,q1_len,q2_len,q1_words,q2 _words,common_words,ttl_words,word_share,cwc_min,cwc_max,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
398782,1,75,76,13,13,12,26,0.462,0.874989,0.874989,...,0.923070,1.0,1.0,0.0,13.0,0.855263,99,99,99,99
115086,0,50,58,13,16,8,24,0.333,0.666644,0.399992,...,0.466664,1.0,1.0,3.0,13.5,0.215686,70,68,62,74
327711,0,106,123,28,21,4,38,0.105,0.000000,0.000000,...,0.115384,0.0,0.0,6.0,23.0,0.046729,26,30,32,38
367788,0,58,145,14,32,1,34,0.029,0.000000,0.000000,...,0.000000,0.0,0.0,17.0,21.5,0.050847,29,41,23,30
151235,0,34,50,5,9,3,13,0.231,0.749981,0.599988,...,0.333330,1.0,0.0,4.0,7.0,0.542857,55,70,45,69


In [33]:
!pip install gensim

In [34]:
from numpy import triu

In [35]:
import gensim

model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [52]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# Ensure you have the stopwords and punkt datasets from nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Function to preprocess and tokenize text
def preprocess(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# Assuming ques_df is your DataFrame containing 'question1' and 'question2' columns
# Example:
# ques_df = pd.DataFrame({
#     'question1': ['What is your name?', 'How are you?'],
#     'question2': ['Where do you live?', 'What is your profession?']
# })

questions = list(ques_df['question1']) + list(ques_df['question2'])

# Preprocess and tokenize the questions
tokenized_questions = [preprocess(question) for question in questions]

# Train Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_questions, vector_size=300, window=5, min_count=1, workers=4)

# Convert tokenized questions back to strings for TF-IDF calculation
questions = [' '.join(tokens) for tokens in tokenized_questions]

# Calculate TF-IDF scores
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(questions)
tfidf_feature_names = vectorizer.get_feature_names_out()

# Create a dictionary mapping words to their TF-IDF scores
tfidf_scores = dict(zip(tfidf_feature_names, vectorizer.idf_))

# Function to convert a question to a vector by averaging weighted word embeddings
def question_to_weighted_vec(question, model, tfidf_scores):
    tokens = preprocess(question)
    vectors = []
    weights = []
    for word in tokens:
        if word in model.wv and word in tfidf_scores:
            vectors.append(model.wv[word] * tfidf_scores[word])
            weights.append(tfidf_scores[word])
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.average(vectors, axis=0, weights=weights)

# Convert each question to a vector
q1_vecs = np.array([question_to_weighted_vec(q, w2v_model, tfidf_scores) for q in ques_df['question1']])
q2_vecs = np.array([question_to_weighted_vec(q, w2v_model, tfidf_scores) for q in ques_df['question2']])

# Now you have q1_vecs and q2_vecs as arrays of vectors
print(q1_vecs.shape, q2_vecs.shape)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arora\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arora\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(30000, 300) (30000, 300)


In [53]:
'''from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from gensim.models import Word2Vec



# Function to preprocess and tokenize text
def preprocess(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# Assuming ques_df is your DataFrame containing 'question1' and 'question2' columns
questions = list(ques_df['question1']) + list(ques_df['question2'])

# Preprocess and tokenize the questions
tokenized_questions = [preprocess(question) for question in questions]

# Train Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_questions, vector_size=300, window=5, min_count=1, workers=4)

# Function to convert a question to a vector by averaging word embeddings
def question_to_vec(question, model):
    tokens = preprocess(question)
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# Convert each question to a vector
q1_vecs = np.array([question_to_vec(q, w2v_model) for q in ques_df['question1']])
q2_vecs = np.array([question_to_vec(q, w2v_model) for q in ques_df['question2']])
'''

"from nltk.tokenize import word_tokenize\nfrom nltk.corpus import stopwords\nimport string\nfrom gensim.models import Word2Vec\n\n# Ensure you have the stopwords and punkt datasets from nltk\nimport nltk\n\n\n# Function to preprocess and tokenize text\ndef preprocess(text):\n    stop_words = set(stopwords.words('english'))\n    tokens = word_tokenize(text.lower())\n    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]\n    return tokens\n\n# Assuming ques_df is your DataFrame containing 'question1' and 'question2' columns\nquestions = list(ques_df['question1']) + list(ques_df['question2'])\n\n# Preprocess and tokenize the questions\ntokenized_questions = [preprocess(question) for question in questions]\n\n# Train Word2Vec model\nw2v_model = Word2Vec(sentences=tokenized_questions, vector_size=300, window=5, min_count=1, workers=4)\n\n# Function to convert a question to a vector by averaging word embeddings\ndef question_to_vec(question, model):\n    tokens 

In [54]:
temp_df1 = pd.DataFrame(q1_vecs, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_vecs, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 600)

In [55]:
final_df = pd.concat([final_df, temp_df], axis=1)
print(final_df.shape)
final_df.head()

(30000, 1223)


,is_duplicate,q1_len,q2_len,q1_words,q2 _words,common_words,ttl_words,word_share,cwc_min,cwc_max,...,290,291,292,293,294,295,296,297,298,299
398782,1,75,76,13,13,12,26,0.462,0.874989,0.874989,...,0.933863,1.200200,1.851302,0.337442,1.561577,2.452665,0.724530,-0.800587,1.311738,-0.584667
115086,0,50,58,13,16,8,24,0.333,0.666644,0.399992,...,0.643054,1.607599,1.981272,0.378257,0.798612,3.045165,0.770287,-0.810967,1.602576,-0.719753
327711,0,106,123,28,21,4,38,0.105,0.000000,0.000000,...,0.830573,0.883170,1.807486,0.060252,1.780334,2.242856,1.272725,-1.001031,1.421049,-0.576508
367788,0,58,145,14,32,1,34,0.029,0.000000,0.000000,...,0.347821,1.171073,1.378573,0.059545,0.562599,2.284448,0.588130,-0.591262,0.792259,-0.453733
151235,0,34,50,5,9,3,13,0.231,0.749981,0.599988,...,0.532987,1.088109,1.897766,-0.252572,1.052277,2.425925,0.773744,-0.688584,0.722872,-0.731374


In [56]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(final_df.iloc[:,1:].values,final_df.iloc[:,0].values,test_size=0.2,random_state=1)

In [57]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7745

In [58]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7688333333333334